## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [311]:
from sqlalchemy import create_engine, text, MetaData, select, func
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [312]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [313]:
### Imprima os três primeiros registros da tabela tracks

tracks = Base.classes.tracks

with Session(engine) as session:
    tracks3 = select(tracks).limit(3)
    lista = list(session.scalars(tracks3))

for track in lista:
    print(f"Id: {track.TrackId}, Nome: {track.Name}, Compositor(es): {track.Composer}, id:{track.AlbumId}")

Id: 1, Nome: For Those About To Rock (We Salute You), Compositor(es): Angus Young, Malcolm Young, Brian Johnson, id:1
Id: 2, Nome: Balls to the Wall, Compositor(es): None, id:2
Id: 3, Nome: Fast As a Shark, Compositor(es): F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman, id:3


In [314]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

albums = Base.classes.albums
with Session(engine) as session:
    tracks20 = select(tracks.Name, albums.Title).join(albums).order_by(tracks.TrackId).limit(20)
    lista = session.execute(tracks20).fetchall()

for track, album in lista:
    print(f"Faixa: {track}, Álbum: {album}")


Faixa: For Those About To Rock (We Salute You), Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall, Álbum: Balls to the Wall
Faixa: Fast As a Shark, Álbum: Restless and Wild
Faixa: Restless and Wild, Álbum: Restless and Wild
Faixa: Princess of the Dawn, Álbum: Restless and Wild
Faixa: Put The Finger On You, Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up, Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom, Álbum: For Those About To Rock We Salute You
Faixa: Snowballed, Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks, Álbum: For Those About To Rock We Salute You
Faixa: C.O.D., Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules, Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives, Álbum: For Those About To Rock We Salute You
Faixa: Spellbound, Álbum: For Those About To Rock We Salute You
Faixa: Go Down, Álbum: Let There Be Rock
Faixa: Dog Eat Dog, Álbum: Let There Be Rock


In [315]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

invoice_items=Base.classes.invoice_items
with Session(engine) as session:
    tracks10 = select(tracks.Name, invoice_items.Quantity).join(tracks).limit(10)
    lista=session.execute(tracks10).fetchall()

for track, quantity in lista:
    print(f"Faixa: {track}, Quantidade: {quantity}")

Faixa: Balls to the Wall, Quantidade: 1
Faixa: Restless and Wild, Quantidade: 1
Faixa: Put The Finger On You, Quantidade: 1
Faixa: Inject The Venom, Quantidade: 1
Faixa: Evil Walks, Quantidade: 1
Faixa: Breaking The Rules, Quantidade: 1
Faixa: Dog Eat Dog, Quantidade: 1
Faixa: Overdose, Quantidade: 1
Faixa: Love In An Elevator, Quantidade: 1
Faixa: Janie's Got A Gun, Quantidade: 1


In [316]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

with Session(engine) as session:
    tracksvend = (select(tracks.Name, func.sum(invoice_items.Quantity).label("total")).join(tracks).group_by(tracks.Name).order_by(func.sum(invoice_items.Quantity).desc()).limit(10))
    lista = session.execute(tracksvend).fetchall() 

for track, vendas in lista:
    print(f"Faixa: {track}, Total Vendido: {vendas}")


Faixa: The Trooper, Total Vendido: 5
Faixa: Untitled, Total Vendido: 4
Faixa: The Number Of The Beast, Total Vendido: 4
Faixa: Sure Know Something, Total Vendido: 4
Faixa: Hallowed Be Thy Name, Total Vendido: 4
Faixa: Eruption, Total Vendido: 4
Faixa: Where Eagles Dare, Total Vendido: 3
Faixa: Welcome Home (Sanitarium), Total Vendido: 3
Faixa: Sweetest Thing, Total Vendido: 3
Faixa: Surrender, Total Vendido: 3


In [317]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

artists = Base.classes.artists

with Session(engine) as session:
    artistavend = (
        select(artists.Name, func.sum(invoice_items.Quantity).label("total_vendas")).join(albums).join(tracks).join(invoice_items)
        .group_by(artists.Name).order_by(func.sum(invoice_items.Quantity).desc()).limit(10) 
    )

    lista = session.execute(artistavend).fetchall()

for artista, total in lista:
    print(f"Artista: {artista}, Total Vendido: {total}")

Artista: Iron Maiden, Total Vendido: 140
Artista: U2, Total Vendido: 107
Artista: Metallica, Total Vendido: 91
Artista: Led Zeppelin, Total Vendido: 87
Artista: Os Paralamas Do Sucesso, Total Vendido: 45
Artista: Deep Purple, Total Vendido: 44
Artista: Faith No More, Total Vendido: 42
Artista: Lost, Total Vendido: 41
Artista: Eric Clapton, Total Vendido: 40
Artista: R.E.M., Total Vendido: 39
